In [ ]:
#objective: create a model to predict when a lot of houses will be put on the market

In [29]:
import pandas as pd
from sklearn import datasets
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest

In [27]:
sales=pd.read_csv(r'C:\Users\Tara\OneDrive - University of Pittsburgh\FALL 2022\ENGR 1171\Project Housing Data\SalesData.csv')
sales['Sold']='Yes'
assessment=pd.read_csv(r'C:\Users\Tara\OneDrive - University of Pittsburgh\FALL 2022\ENGR 1171\Project Housing Data\AssessmentData.csv')

C:\Users\Tara\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\Tara\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (19,20,29,30,37,38,46,57,83) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
df=pd.merge(assessment,sales,how='left',on=['PARID','PROPERTYHOUSENUM','PROPERTYFRACTION','PROPERTYCITY','PROPERTYSTATE','PROPERTYZIP',
                                            'SCHOOLCODE','MUNICODE','MUNIDESC'])
df['Sold']=df['Sold'].fillna('No')

In [33]:
x=df.drop('Sold',axis=1)
y=df[['Sold']]

In [40]:
df.to_csv('mergeddata.csv')

In [38]:
chi2_features = SelectKBest(chi2, k = 10)
X_kbest_features = chi2_features.fit_transform(x, y)
print('Original feature number:', x.shape[1])
print('Reduced feature number:', X_kbest.shape[1])

ValueError: could not convert string to float: '0001G00104000000'